In [1]:
import pandas as pd
import numpy as np
from readFile import split_into_values, toRPdata

In [2]:
# columns 와 value는 사용자 입력
df = pd.read_csv('resources/AXISX_resample.csv')
#df = pd.read_csv('resources/CLAMP_resample.csv')
value = ['value']
#df = pd.read_csv('resources/Dataset1.csv')
columns = ['chip', 'wire', 'segment']
value = ['value']

In [3]:
df = df.loc[:, columns + value] #('chip', 'wire', 'value')는 사용자 입력
size = 96 # 모델에 따라 변경 28, 96
result = split_into_values(df, columns)

In [4]:
# dataframe -> list
result_list = result.values.tolist()

In [5]:
from sklearn.preprocessing import MinMaxScaler

def MinMax(data):
    MMS = MinMaxScaler().fit(data)
    scaled = MMS.transform(data)
    return scaled

In [6]:
# result_list transpose
result_T = [list(x) for x in zip(*result_list)]

In [7]:
# minmax 정규화
result_scaled = MinMax(result_T)

In [8]:
# 다시 result transpose 해서 원래대로
result_scaled = [list(x) for x in zip(*result_scaled)]

In [9]:
from tslearn.preprocessing import TimeSeriesScalerMeanVariance, TimeSeriesResampler

In [10]:
# 2. 시계열 셋 크기 변경
result_ = TimeSeriesResampler(sz=size).fit_transform(result)

In [11]:
data = result_.reshape(result_.shape[0], 1, size)
#X = toRPdata(data, threshold='point', percentage=30) # rp 1
X = toRPdata(data) # rp 2

In [12]:
X_scaled = np.expand_dims(X, axis=3)
X_scaled.shape

(1330, 96, 96, 1)

In [13]:
from keras.models import load_model
model = load_model('rp2_AXIXS_96_Autoencoder_128_3e-05_weight.h5')

In [14]:
print(model.summary())

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_35 (Conv2D)           (None, 96, 96, 32)        320       
_________________________________________________________________
activation_35 (Activation)   (None, 96, 96, 32)        0         
_________________________________________________________________
max_pooling2d_15 (MaxPooling (None, 48, 48, 32)        0         
_________________________________________________________________
conv2d_36 (Conv2D)           (None, 48, 48, 16)        4624      
_________________________________________________________________
activation_36 (Activation)   (None, 48, 48, 16)        0         
_________________________________________________________________
max_pooling2d_16 (MaxPooling (None, 24, 24, 16)        0         
_________________________________________________________________
conv2d_37 (Conv2D)           (None, 24, 24, 2)        

In [15]:
from keras import backend as K

compressed_layer = 8
get_layer_output = K.function([model.layers[0].input],[model.layers[compressed_layer].output])
compressed = get_layer_output([X_scaled])[0]
feature = compressed.reshape(compressed.shape[0], compressed.shape[1]*compressed.shape[2]*compressed.shape[3])

print(feature)
print(feature.shape)

[[1.1367555  0.26484057 2.505173   ... 0.13618654 1.3474734  0.40889412]
 [2.220899   0.         3.950424   ... 0.20262879 0.7216671  0.        ]
 [3.848504   1.1673617  4.2500215  ... 0.         2.7189982  0.        ]
 ...
 [3.3162796  0.14355874 5.060283   ... 0.         1.82039    0.        ]
 [1.8259916  0.5287815  4.9768133  ... 0.4699223  0.5744387  0.7971715 ]
 [2.109472   0.6446901  4.6462035  ... 0.         2.874545   0.        ]]
(1330, 288)


In [ ]:
import numpy

numpy.save('./compressed2', compressed) # x_save.npy
numpy.save('./feature2', feature) # x_save.npy
numpy.save('./X_scaled2', X_scaled) # x_save.npy